In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import albumentations as A
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from tensorflow import keras
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Lambda,BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [3]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose
)

In [4]:
tf.__version__

'2.1.0-rc2'

### Some preprocessing helping func's

In [5]:
def augment(aug,image):
    image = aug(image=image)['image']
    return image
    
def augment_and_show(aug,image):
    image = augment(aug,image)
    plt.figure(figsize=(10,10))
    plt.imshow(image)

In [6]:
def augment_2d(inputs,p=.5):
    return augment(Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ], p=p),inputs)
    

In [7]:
def find_in_dir(dirname):
    return [os.path.join(dirname, fname) for fname in sorted(os.listdir(dirname))]


In [8]:
train_datagen = ImageDataGenerator()

train_generator = ImageDataGenerator().flow_from_directory(
'images',
target_size = (256,256),
batch_size = 128)

Found 194928 images belonging to 39 classes.


In [9]:
train_generator.image_shape

(256, 256, 3)

In [10]:
model = Sequential()
model.add(Lambda(augment_2d,input_shape=train_generator.image_shape))
model.add(Conv2D(3,6,5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(6,16,5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(120))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(84))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(39))
model.add(Activation('softmax'))

NotImplementedError: Cannot convert a symbolic Tensor (lambda_input:0) to a numpy array.

In [ ]:
opt = tf.keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
             optimizer=opt,
             metrics=['accuracy'])
model.fit_generator(train_generator,epochs=10)